In [169]:
import math

In [170]:
class Position:
    def __init__(self, table, player, opponent, parent) -> None:
        self.table_tokens = table
        self.player_tokens = player
        self.opponent_tokens = opponent
        self.legal_moves = self.set_legal_moves()
        self.cost = None
        self.parent = parent
        self.children = []

    def set_legal_moves(self):
        if self.table_tokens >= 3:
            return [3, 2, 1]
        elif self.table_tokens == 2:
            return [2, 1]
        elif self.table_tokens == 1:
            return [1]
        else:
            return []

    def is_terminal_position(self):
        return True if self.table_tokens == 0 else False

    def set_cost(self, cost):
        self.cost = cost

    def next_best_move(self):
        for each_child in self.children:
            if each_child.cost == self.cost:
                return each_child

    def __str__(self) -> str:
        return f'[T: {self.table_tokens}, P: {self.player_tokens}, O: {self.opponent_tokens}, C: {self.cost}]'

In [190]:
def heuristic(position: Position, is_max_round: bool, n):
    if position.table_tokens == 0 and is_max_round:
        return 1
    elif position.table_tokens == 0 and not is_max_round:
        return -1
    else:
        return 0

In [172]:
def minimax(position : Position, depth : int, alpha : float, beta : float, is_max_round : bool, heuristic : callable, n):

    if depth == 0 or position.is_terminal_position():
        cost = heuristic(position, is_max_round, n)
        position.set_cost(cost)
        return cost

    if is_max_round:
        max_eval = -math.inf
        for k in position.legal_moves:
            child_position = Position(position.table_tokens - k, position.player_tokens + k, position.opponent_tokens, position)
            position.children.append(child_position)
            eval = minimax(child_position, depth - 1, alpha, beta, False, heuristic, n)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        position.set_cost(max_eval)
        return max_eval
    else:
        min_eval = math.inf
        for k in position.legal_moves:
            child_position = Position(position.table_tokens - k, position.player_tokens, position.opponent_tokens + k, position)
            position.children.append(child_position)
            eval = minimax(child_position, depth - 1, alpha, beta, True, heuristic, n)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        position.set_cost(min_eval)
        return min_eval


In [176]:
class Game:
    def __init__(self, n, max_depth, min_depth, is_max_round, heuristic) -> None:
        self.current_position = Position(n, 0, 0, None)
        self.max_depth = max_depth
        self.min_depth = min_depth
        self.is_max_round = is_max_round
        self.heuristic = heuristic
        self.n = n

    def play(self):

        while self.current_position.table_tokens != 0:

            if self.is_max_round:
                minimax(self.current_position, self.max_depth, -math.inf, math.inf, True, self.heuristic, self.n)
                self.is_max_round = False
            else:
                minimax(self.current_position, self.min_depth, -math.inf, math.inf, False, self.heuristic, self.n)
                self.is_max_round = True

            self.current_position = self.current_position.next_best_move()

        if self.is_max_round:
            return 'Max won!'
        else:
            return 'Min won!'

In [200]:
n = 15
max_depth = 5
min_depth = 10
is_max_round = True
heuristic = heuristic
results = []
for _ in range(100):
    game = Game(n, max_depth, min_depth, is_max_round, heuristic)
    results.append(game.play())

print("Max won " + str(results.count('Max won!')) + " times")
print("Min won " + str(results.count('Min won!')) + " times")

Max won 0 times
Min won 100 times
